In [1]:
setwd("~/0-workspace/CCR7_DC/TC_all_LN/")

library(Seurat) # v4.4
library(Matrix)
library(future)
library(dplyr)
library(ggplot2)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(ggrepel)

set.seed(1)
options(future.globals.maxSize = Inf)

pal <- readRDS("plots/palette.rds")

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 

In [2]:
RenameGenesSeurat <- function(obj, newnames) { # Replace gene names in different slots of a Seurat object. Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.
  print("Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.")
  RNA <- obj@assays$RNA
  
  if (nrow(RNA) == length(newnames)) {
    if (length(RNA@counts)) RNA@counts@Dimnames[[1]] <- newnames
    if (length(RNA@data)) RNA@data@Dimnames[[1]] <- newnames
    if (length(RNA@scale.data)) rownames(RNA@scale.data) <- newnames
    if (length(RNA@meta.features)) {
      RNA@meta.features$prev.names <- rownames(RNA@meta.features)
      rownames(RNA@meta.features) <- newnames
    }
    if (length(RNA@var.features)){
      RNA@var.features <- rownames(RNA@meta.features[match(RNA@var.features, RNA@meta.features$prev.names), ])
    }
  } else {"Unequal gene sets: nrow(RNA) != nrow(newnames)"}
  obj@assays$RNA <- RNA
  return(obj)
}

In [6]:
select.markers <- function(fn, output.dir = 'results/markers/', pairwise = F, fc.thr = 1.5, apv.thr = 0.01, n = Inf, exCC = F, cc.genes = NULL){
  markers <- read.csv(file = paste0(output.dir, fn, ".csv"), header = T, stringsAsFactors = F) %>%
    subset(!grepl("(^MT-)|(^RPS)|(^RPL)|(^MRPL)|(^MRPS)", toupper(gene)))
  if (exCC){
    markers <- markers[!(toupper(markers$gene) %in% toupper(cc.genes)),]
  }
  if(pairwise){
    markers <- subset(markers, abs(avg_log2FC) > log2(fc.thr) & p_val_adj < apv.thr)
    markers$cluster <- ifelse(markers$avg_log2FC > 0, "up", "down")
    if(!is.infinite(n)) markers <- markers %>% group_by(cluster) %>% top_n(n, abs(avg_log2FC))
    markers <- markers[order(markers$avg_log2FC), ]
  } else{
    markers <- subset(markers, avg_log2FC > log2(fc.thr) & p_val_adj < apv.thr)
    if(any(duplicated(markers$gene))) markers <- markers %>% group_by(gene) %>% top_n(1, avg_log2FC)
    if(!is.infinite(n)) markers <- markers %>% group_by(cluster) %>% top_n(n, avg_log2FC)
    markers <- markers[order(markers$cluster, -markers$avg_log2FC), ]
  }
  return(markers)
}

In [26]:
plot.continuous.value <- function(
    sro = NULL, idx = NULL,
    vis = sro@reductions$umap@cell.embeddings,
    val, val.name,
    scale.color=scale_color_distiller(palette = "Spectral"), point.size = 1
){
  ggplot() +
    geom_point(
      mapping = aes(x = vis[idx, 1], y = vis[idx, 2], color = val[idx]),
      size = point.size, alpha = 0.8
    ) +
    scale.color + theme_classic() + labs(color = val.name)+
    theme(axis.ticks = element_blank(), axis.text = element_blank(), axis.title = element_blank())
}

get.named.vector.sro <- function(sro, colname){
  val.vec <- sro@meta.data[, colname]; names(val.vec) <- rownames(sro@meta.data)
  return(val.vec)
}


# DC module scores from Colonna data

In [3]:
pref.sro <- "results/"; pref.p.sro <- "plots/"
sro <- readRDS(paste0(pref.sro, "SRO_subset.rds"))

In [4]:
head(rownames(sro))

[1] "XKR4"   "GM1992" "MRPL15" "LYPLA1" "TCEA1"  "RGS20"

## Load markers

In [7]:
# load markers ####
markers.dc <- select.markers("RNA_snn_res.1.8-exCCR7DC1_cDC1_Ki67DC1", 
                             output.dir = paste0('../Colonna/results/scrna/merged/subset/markers/'), exCC = F)

In [8]:
md <- read.csv("../Colonna/results/scrna/merged/subset/meta-data.csv", row.names = 1)

In [9]:
md2 <- unique(md[c('RNA_snn_res.1.8', 'annotation')]) %>% arrange(RNA_snn_res.1.8)

In [15]:
md2[md2$annotation == "tDC",]$RNA_snn_res.1.8

[1] 13 17 23 32 33

In [16]:
tdc.markers <- markers.dc[markers.dc$cluster %in% md2[md2$annotation == "tDC",]$RNA_snn_res.1.8,]

In [17]:
tdc.marker.genes <- toupper(unique(tdc.markers$gene))
tdc.marker.genes <- tdc.marker.genes[tdc.marker.genes %in% rownames(sro)]

In [18]:
length(tdc.marker.genes)

[1] 796

In [19]:
sro <- AddModuleScore(sro, list(toupper(tdc.marker.genes)), name = "tDC.markers")

In [20]:
cols <- rev(RColorBrewer::brewer.pal(11,"Spectral"))
transitions <- c(0, 15, 25, 30, 40, 50, 60, 70, 75, 85, 100)
scaled_transitions <- scales::rescale(transitions, from = c(0, 100), to = c(0, 1))
scale.color <- scale_color_gradientn(colors = cols, values = scaled_transitions)

In [21]:
dir.create(paste0(pref.p.sro, "module-scores"))

Warning message in dir.create(paste0(pref.p.sro, "module-scores")):
“'plots/module-scores' already exists”


In [22]:
pref.p.sro

[1] "plots/"

In [27]:
pdf(paste0(pref.p.sro, "module-scores/UMAP-tDC.markers-from-Colonna_scRNA.pdf"), width = 8, height = 6)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    idx = rownames(sro@meta.data), point.size = 0.5,
    val = get.named.vector.sro(sro, paste0("tDC.markers1")), val.name = 'module\nscore') + ggtitle("tDC.markers"))
dev.off()

png 
  2

# sub0 from C15 in res2-opt2 from Flex-seq (CB-3141)

In [4]:
pref.sro <- "results/"; pref.p.sro <- "plots/"
sro <- readRDS(paste0(pref.sro, "SRO_subset.rds"))

In [5]:
head(rownames(sro))

[1] "XKR4"   "GM1992" "MRPL15" "LYPLA1" "TCEA1"  "RGS20"

## Load markers

In [46]:
group.name <- "RNA_snn_res.1.2_C15res.0.1"
fn <- paste0(group.name, "-sub0_vs_rest")

In [47]:
fn

[1] "RNA_snn_res.1.2_C15res.0.1-sub0_vs_rest"

In [48]:
# load markers ####
markers <- select.markers(fn, output.dir = paste0('../CB-3141_pool/results/res-2-subset-progenitors-option2/markers/'), exCC = F, pairwise = T)

In [49]:
sub0markers <- markers[markers$cluster == 'up',]

In [50]:
nrow(sub0markers)

[1] 321

In [20]:
sro <- AddModuleScore(sro, list(toupper(sub0markers$gene)), name = "T15.sub0.markers")

In [21]:
cols <- rev(RColorBrewer::brewer.pal(11,"Spectral"))
transitions <- c(0, 15, 25, 30, 40, 50, 60, 70, 75, 85, 100)
scaled_transitions <- scales::rescale(transitions, from = c(0, 100), to = c(0, 1))
scale.color <- scale_color_gradientn(colors = cols, values = scaled_transitions)

In [22]:
dir.create(paste0(pref.p.sro, "module-scores"))

Warning message in dir.create(paste0(pref.p.sro, "module-scores")):
“'plots/module-scores' already exists”


In [23]:
pref.p.sro

[1] "plots/"

In [25]:
pdf(paste0(pref.p.sro, "module-scores/UMAP-T15.sub0.markers-from-CB-3141.pdf"), width = 8, height = 6)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    idx = rownames(sro@meta.data), point.size = 0.5,
    val = get.named.vector.sro(sro, paste0("T15.sub0.markers1")), val.name = 'module\nscore') + ggtitle("T15.sub0.markers"))
dev.off()

png 
  2

# sub0 from C15 in res2-opt2 from Flex-seq (CB-3141)

sub0 vs sub1 and C19 (TC)

In [4]:
pref.sro <- "results/"; pref.p.sro <- "plots/"
sro <- readRDS(paste0(pref.sro, "SRO_subset.rds"))

In [26]:
head(rownames(sro))

[1] "XKR4"   "GM1992" "MRPL15" "LYPLA1" "TCEA1"  "RGS20"

## Load markers

In [53]:
group.name <- "RNA_snn_res.1.2_C15res.0.1"
fn <- paste0(group.name, "-sub0_vs_sub0_C19")
fn

[1] "RNA_snn_res.1.2_C15res.0.1-sub0_vs_sub0_C19"

In [43]:
# load markers ####
markers <- select.markers(fn, output.dir = paste0('../CB-3141_pool/results/res-2-subset-progenitors-option2/markers/'), exCC = F, pairwise = T)
sub0markers <- markers[markers$cluster == 'up',]

In [59]:
# load markers ####
markers <- select.markers(fn, output.dir = paste0('../CB-3141_pool/results/res-2-subset-progenitors-option2/markers/'), exCC = F, pairwise = T)
sub0markers <- markers[markers$cluster == 'up',]

In [60]:
nrow(sub0markers)

[1] 164

In [61]:
sro <- AddModuleScore(sro, list(toupper(sub0markers$gene)), name = "T15.sub0_vs_sub1_C19.markers")

In [62]:
pdf(paste0(pref.p.sro, "module-scores/UMAP-T15.sub0_vs_sub1_C19.markers-from-CB-3141.pdf"), width = 8, height = 6)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    idx = rownames(sro@meta.data), point.size = 0.5,
    val = get.named.vector.sro(sro, paste0("T15.sub0_vs_sub1_C19.markers1")), val.name = 'module\nscore') + ggtitle("T15.sub0.markers"))
dev.off()

png 
  2

In [54]:
# load markers ####
markers <- select.markers(fn, output.dir = paste0('../CB-3141_pool/results/res-2-subset-progenitors-option2/markers/'), exCC = F, pairwise = T, fc.thr = 2)
sub0markers <- markers[markers$cluster == 'up',]

In [55]:
nrow(sub0markers)

[1] 45

In [56]:
sro <- AddModuleScore(sro, list(toupper(sub0markers$gene)), name = "T15.sub0_vs_sub1_C19.markers")

In [58]:
pdf(paste0(pref.p.sro, "module-scores/UMAP-T15.sub0_vs_sub1_C19.markers-from-CB-3141-v2.pdf"), width = 8, height = 6)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    idx = rownames(sro@meta.data), point.size = 0.5,
    val = get.named.vector.sro(sro, paste0("T15.sub0_vs_sub1_C19.markers1")), val.name = 'module\nscore') + ggtitle("T15.sub0.markers"))
dev.off()

png 
  2

In [67]:
pdf(paste0(pref.p.sro, "module-scores/vln-T15.sub0_vs_sub1_C19.markers-from-CB-3141-v2.pdf"), width = 8, height = 6)
VlnPlot(object = sro, features = 'T15.sub0_vs_sub1_C19.markers1', group.by = "Annotation", cols = pal$Annotation)
dev.off()


png 
  2